In [17]:
import numpy as np
import pandas as pd


In [18]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [21]:
train_data[train_data['target'] == 1]['text'].sample().values

array(['Fears over missing migrants in Med: Rescuers search for survivors after a boat carrying as many as 600 migrants\x89Û_ http://t.co/v9ftYB30EI'],
      dtype=object)

In [22]:
train_data[train_data.text.str.contains('', case=False)].target.value_counts()


0    4342
1    3271
Name: target, dtype: int64

##### True Positive tweets tend to result in multiple tweets when searched in twitter

In [11]:
## Feature ideas
### All caps
### Ausrufezeichen
### Youtube
### Internet links
train_data[train_data.text.str.contains('http')].target.value_counts() # Mehr wahrscheinlichkeit von Disaster tweet
### @ 
train_data[train_data.text.str.contains('@', case = False)].target.value_counts() # Entries with @ tends to be less serious
### Text Containing news
train_data[train_data.text.str.contains('news', case = False)].target.value_counts() 
### I sentences
train_data[train_data.text.str.contains('i ', case=False)].target.value_counts()




0    1030
1     456
Name: target, dtype: int64

## Define keyword feature generator method

In [3]:
def keyword_feature_generator(keyword, feature_name, data = train_data):
    data[feature_name] = 0
    idx = data[data.text.str.contains(keyword, case=False)].index
    train_data.loc[idx, feature_name] = 1

## Add I sentence feature

In [5]:
keyword = 'i '
keyword_feature_generator(keyword, 'i_sentence')

## Add http feature

In [7]:
keyword = 'http'
keyword_feature_generator(keyword, 'link_attached')

## Add @ feature

In [51]:
train_data[train_data.text.str.contains('@')].target.value_counts() 

0    1363
1     676
Name: target, dtype: int64

In [9]:
keyword = '@'
keyword_feature_generator(keyword, 'at_in_text')

## Add news feature

In [10]:
keyword = "news"
keyword_feature_generator(keyword, 'news_in_text')